<a href="https://colab.research.google.com/github/HechengJin0/MGC_paper/blob/master/paper_monkey_figures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# run in psc server

In [ ]:
# from hyppo.independence import Dcorr
from hyppo.independence import MGC
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from scipy.stats import pearsonr, zscore
import seaborn as sns
import csv
import pickle
import matplotlib.pyplot as plt
import nibabel as nib
import cifti
from nilearn import surface
from nilearn import plotting

# 1.read mgc result

In [ ]:
subfile_path = '/monkey/scan120_data.list'
sub_list = pd.read_csv(subfile_path, delimiter='\t', header=None, names=['sub-file'])
subfile_path = '/monkey/scan120_state.list'
state_list = pd.read_csv(subfile_path, delimiter='\t', header=None, names=['state'])
monkey_120_df = sub_list.merge(state_list, left_index=True, right_index=True)

In [ ]:
monkey_name = 'monkey_M' # monkey_M monkey_R monkey_B
isolevels = [0,75,100,150,200]
p=91

# 5 state * 12/8/4 session
test_statistic_all = []
pearson_corr_all = []
optimal_scale_X_all = []
optimal_scale_X_nonlinear_binary_all = []

for i, isolevel in enumerate(isolevels):

    test_statistic_each_state = []
    pearson_corr_each_state = []
    optimal_scale_X_each_state = []
    optimal_scale_X_nonlinear_binary_each_state = []
    state_df = monkey_120_df[(monkey_120_df['sub-name']==monkey_name) & (monkey_120_df['state']==isolevel)]

    for file_i in range(len(state_df)):
        filename = state_df['sub-file'].iloc[file_i]+'.pkl'

        tr=240
        threshold_nonlinear=0.95

        with open(filename, 'rb') as pkl_file:
            results = pickle.load(pkl_file)

        test_statistic = np.zeros((p,1))
        pearson_corr = np.zeros((p,1))
        optimal_scale_X = np.zeros((p,1))
        optimal_scale_X_nonlinear_binary = np.zeros((p,1))

        for result_i in results:
            k = int(result_i[0])
            test_statistic[k,:] = result_i[1]
            optimal_scale_X[k,:] = result_i[2][0]
            pearson_corr[k,:] = result_i[3]

            if result_i[2][0] / tr >= threshold_nonlinear:
                optimal_scale_X_nonlinear_binary[k,:] = 0
            else:
                optimal_scale_X_nonlinear_binary[k,:] = 1

        test_statistic_each_state.append(test_statistic)
        pearson_corr_each_state.append(pearson_corr)
        optimal_scale_X_each_state.append(optimal_scale_X)
        optimal_scale_X_nonlinear_binary_each_state.append(optimal_scale_X_nonlinear_binary)

    test_statistic_all.append(test_statistic_each_state)
    pearson_corr_all.append(pearson_corr_each_state)
    optimal_scale_X_all.append(optimal_scale_X_each_state)
    optimal_scale_X_nonlinear_binary_all.append(optimal_scale_X_nonlinear_binary_each_state)

In [ ]:
test_statistic_all_state_mean = []
pearson_corr_all_state_mean = []
optimal_scale_X_all_state_mean = []
optimal_scale_X_nonlinear_binary_all_state_mean = []

for i, isolevel in enumerate(isolevels):
    test_statistic_all_state_mean.append(np.mean(test_statistic_all[i],axis=0))
    pearson_corr_all_state_mean.append(np.mean(pearson_corr_all[i],axis=0))
    optimal_scale_X_all_state_mean.append(np.mean(optimal_scale_X_all[i],axis=0))
    optimal_scale_X_nonlinear_binary_all_state_mean.append(np.mean(optimal_scale_X_nonlinear_binary_all[i],axis=0))

np.shape(optimal_scale_X_nonlinear_binary_all)

(5, 8, 91, 1)

# 2.plot on hemi

In [ ]:
def parcel2vert_r(plabel,theta_img):
    glasserlabel,(ax1,ax2) = cifti.read(plabel)
    numverts = int(glasserlabel.shape[1]/2)
    if len(theta_img.shape) >1:
        numparcels = len(theta_img)
    else:
        numparcels = 1
        theta_img = np.reshape(theta_img,[-1,len(theta_img)])
    nparc = np.arange(numparcels)
    data = np.zeros([numparcels,numverts])
    for parcel in nparc:
        for plabel in range(int(np.max((glasserlabel)/2)), int(np.max((glasserlabel)))):
            p_idx = np.where(glasserlabel[0,:]==plabel+1)
            plabel = plabel-91 #hardcode!!
            data[parcel,p_idx] = theta_img[parcel,plabel]
    return data


In [ ]:
parcel_label = '/monkey/parcels/MarkovCC12_M132_182-area.10k_fs_LR.dlabel.nii'
isolevels = [0,75,100,150,200]
ROI_Limg = nib.load('/monkey/parcels/L.atlasroi.10k_fs_LR.shape.gii')
L_imagemask = np.array(ROI_Limg.darrays[0].data)
ROI_Rimg = nib.load('/monkey/parcels/R.atlasroi.10k_fs_LR.shape.gii')
R_imagemask = np.array(ROI_Rimg.darrays[0].data)
L_data_mask = np.where(L_imagemask == 1)
R_data_mask = np.where(R_imagemask == 1)

bg_map = "/monkey/parcels/MAJOM.L.midthickness.10k_fs_LR.surf.gii"

rsurf = surface.load_surf_mesh('/monkey/parcels/MAJOM.R.inflated.10k_fs_LR.surf.gii')
lsurf = surface.load_surf_mesh('/monkey/parcels/MAJOM.L.inflated.10k_fs_LR.surf.gii')

vmat10k_r_all_state = []
for i, isolevel in enumerate(isolevels):
    vmat10k_r = np.zeros([10242])
    vmat10k_r[R_data_mask] = parcel2vert_r(parcel_label,np.array((optimal_scale_X_nonlinear_binary_all_state_mean[i]).reshape(1,91)))
    vmat10k_r_all_state.append(vmat10k_r)

plt.figure(figsize=(10,30))
views = ['lateral', 'medial', 'dorsal']
bg_map = '/monkey/parcels/lh.MacaqueYerkes19.curvature.10k_fs_LR.nii.gz'

cmap = 'gist_heat_r'#'RdBu_r' gist_heat_r
vmax = 1
vmin = 0

# 5 state * 2 view - homotopic
j=1
for i, isolevel in enumerate(isolevels):
    ax = plt.subplot(5,2,2*j-1, projection='3d') #bg_on_data=False, when plot pearson, R, optimal scale, True, when plot threshold, nonlinear
    plotting.plot_surf(lsurf, vmat10k_r_all_state[i], bg_map=bg_map, bg_on_data=True,hemi='left',view = 'lateral', cmap = cmap, colorbar = False, vmin = vmin, vmax = vmax, avg_method = 'median', alpha = 1, darkness = 0.5, symmetric_cmap=False,axes=ax,threshold=None)

    ax = plt.subplot(5,2,2*j, projection='3d')
    plotting.plot_surf(lsurf, vmat10k_r_all_state[i], bg_map=bg_map, bg_on_data=True, hemi='left',view = 'medial', cmap = cmap, colorbar = False, vmin = vmin, vmax = vmax, avg_method = 'median', alpha = 1, darkness = 0.5, symmetric_cmap=False,axes=ax,threshold=None)

    j+=1

plt.tight_layout()
plt.show()
